<a href="https://colab.research.google.com/github/FerreiraBX95/Master-Thesis---Bruno-Xavier-Ferreira/blob/Development-of-models-for-measurement-of-pH-in-atmospheric-and-pressurized-systems-using-artificial-intelligent-strategies/Code_for_training_the_classification_CNN_models_Input_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***  Code for training the classification CNN models - Input 1***

Inicialização do Weights and Bias (WandB)

In [ ]:
# WandB – Install the W&B library
%pip install wandb -q
#!pip install wandb --upgrade
import wandb
from wandb.keras import WandbCallback

Informações da GPU a ser utilizada

In [ ]:
# Para usar a GPU eh preciso antes "Alterar o tipo de ambiene de Execução" para GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print('Informacao da GPU: ')
  print(gpu_info)

Atualizações de programas (se necessario)

In [ ]:
%pip install scikit-plot -q

In [ ]:
%pip install scikit-learn -q

In [ ]:
#!pip install keras --upgrade
#!pip install tensorflow --upgrade
#!pip install sklearn --upgrade


In [ ]:
%pip install keras-metrics -q

Bibliotecas

In [ ]:
# Bibliotecas

import os
import random

import cv2
from matplotlib import image
from matplotlib import pyplot

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import   Conv2D
from tensorflow.keras.layers import   MaxPooling2D
from tensorflow.keras.layers import   GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras import metrics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score 
from sklearn.metrics import f1_score 
from sklearn.metrics import fbeta_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 

from skimage.transform import resize
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_confusion_matrix

import warnings
warnings.filterwarnings('ignore')

Carregamento e pre-processamento dos dados

In [ ]:
## Teste para leitura de imagens do drive
address = '/content/drive/MyDrive/Mestrado_Bruno Xavier/Dissertacao de Mestrado - Cod/Banco_de_Imagens_Curva_de_calib_PYTHON_med_pH_Alline_ajust'
CATEGORIES = ["pH_2","pH_3","pH_4","pH_5","pH_6","pH_7","pH_8","pH_9","pH_10"]

tam= 313
x_db = np.zeros((tam,800,1280,3),np.int16)
y_db = np.zeros((tam,1),np.int16)

ind = 0
for category in CATEGORIES:  
    path = os.path.join(address,category)  # create path
    for img in os.listdir(path):  # iterate over each image
        img_cap = image.imread(os.path.join(path,img))  # convert to array
        if category == 'pH_2':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 0
          #ind = ind + 1
        if category == 'pH_3':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 1
          #ind = ind + 1
        if category == 'pH_4':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 2
          #ind = ind + 1
        if category == 'pH_5':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 3
          #ind = ind + 1
        if category == 'pH_6':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 4
          #ind = ind + 1
        if category == 'pH_7':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 5
          #ind = ind + 1
        if category == 'pH_8':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 6
          #ind = ind + 1
        if category == 'pH_9':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 7
          #ind = ind + 1
        if category == 'pH_10':
          x_db[ind,:,:,:] = img_cap
          y_db[ind] = 8
          #ind = ind + 1
        ind = ind + 1


In [ ]:
print('Tam vetor x: ',len(x_db))
print('Tam vetor y: ',len(y_db))
#print(y_db)

Tam vetor x:  313
Tam vetor y:  313


In [ ]:
# Pode ser preciso rodar duas vezez
x_train, x_test, y_train, y_test = train_test_split(x_db, y_db, test_size=0.3,random_state=1)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5,random_state=1)

In [ ]:
print('Tam vetor y_train: ',len(y_train))
print('Tam vetor y_train: ',len(y_test))
print('Tam vetor y_val: ',len(y_val))
print('Tam vetor y: ',len(y_db))
print('Tam vetor y_som: ',len(y_train)+len(y_test)+len(y_val))

Contagem de itens por classe

In [ ]:
print("counts of label '0' (pH=2): {} \n".format(sum(y_db == 0)))
print("counts of label '1' (pH=3): {}".format(sum(y_db == 1))) 
print("counts of label '2' (pH=4): {}".format(sum(y_db == 2))) 
print("counts of label '3' (pH=5): {}".format(sum(y_db == 3))) 
print("counts of label '4' (pH=6): {}".format(sum(y_db == 4))) 
print("counts of label '5' (pH=7): {}".format(sum(y_db == 5))) 
print("counts of label '6' (pH=8): {}".format(sum(y_db == 6))) 
print("counts of label '7' (pH=9): {}".format(sum(y_db == 7))) 
print("counts of label '8' (pH=10): {}".format(sum(y_db == 8)))

Função para a criacao do modelo CNN

In [ ]:
def Model(filt_siz_1,filt_siz_2,ker_siz_1,ker_siz_2,
          n_layers,p_dropout,siz_dense_1,siz_dense_2):

  inputs = keras.layers.Input(shape=(800, 1280, 3))

  if n_layers==2:
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation='relu')(inputs)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    x = keras.layers.Conv2D(filters=filt_siz_2, kernel_size=(ker_siz_2,ker_siz_2), activation='relu')(x)
  if n_layers==3:
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation='relu')(inputs)
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation='relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    x = keras.layers.Conv2D(filters=filt_siz_2, kernel_size=(ker_siz_2,ker_siz_2), activation='relu')(x)
  if n_layers==4:
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation='relu')(inputs)
    x = keras.layers.Conv2D(filters=filt_siz_1, kernel_size=(ker_siz_1,ker_siz_1), activation='relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    x = keras.layers.Conv2D(filters=filt_siz_2, kernel_size=(ker_siz_2,ker_siz_2), activation='relu')(x)
    x = keras.layers.Conv2D(filters=filt_siz_2, kernel_size=(ker_siz_2,ker_siz_2), activation='relu')(x)

  x = keras.layers.GlobalAveragePooling2D()(x)

  x = keras.layers.Dense(siz_dense_1, activation='relu')(x)
  x = keras.layers.Dropout(p_dropout)(x)
  x = keras.layers.Dense(siz_dense_2, activation='relu')(x)
  
  outputs = keras.layers.Dense(len(CATEGORIES), activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

Treinamento do modelo com sweep wandb

In [ ]:
project_name="sweeps-CNN_pH_teste_37-med-pH-Alline-ajust"

In [ ]:
def train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'epochs': 60,
        'batch_size': 8,
        'n_layers': 2,
        'filters_size_1':8,
        'filters_size_2':8,
        'kernel_size_1': 3,
        'kernel_size_2': 3,
        'p_dropout':0.2,
        'learning_rate': 0.001,
        'siz_dense_1':128,
        'siz_dense_2':32
    }

    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)

    ## Prepare trainloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # prepare valloader 
    valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    valloader = valloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # prepare testloader 
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    # Iniialize model with hyperparameters
    keras.backend.clear_session()
    model = Model(wandb.config.filters_size_1,wandb.config.filters_size_2, wandb.config.kernel_size_1,wandb.config.kernel_size_2,
                  wandb.config.n_layers,wandb.config.p_dropout,wandb.config.siz_dense_1,wandb.config.siz_dense_2)
    
    model.summary()
    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate) # optimizer with different learning rate specified by config
    model.compile(opt, loss='sparse_categorical_crossentropy', metrics=['acc','accuracy'])

    # Train the model
    treinamento = model.fit(trainloader, epochs=wandb.config.epochs, validation_data= valloader, callbacks=[WandbCallback()]) # WandbCallback to automatically track metrics

    # Evaluate    
    #loss, accuracy = model.evaluate(testloader, callbacks=[WandbCallback()])
    #print('Test Error Rate: ', round((1-accuracy)*100, 2))
    #wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)}) # wandb.log to track custom metrics

    predictions = model.predict(x_test)
    wandb.log({"ROC Curve" : wandb.plot.roc_curve(y_test,predictions, labels=CATEGORIES)})

    class_score_data = []
    for test, pred in zip(y_test, predictions):
      class_score_data.append([test, pred])
    wandb.log({"class_scores": wandb.Table(data=class_score_data,columns=["test", "pred"])})
    
    # Log Confusion Matrix
    y_test_pred_class = np.argmax(predictions, axis=1)
    fig, ax = plt.subplots(figsize=(16, 12))
    #plot_confusion_matrix(y_test, y_test_pred_class,ax=ax) #scikitplot
    plot_confusion_matrix(y_test, y_test_pred_class,ax=ax,normalize ='True') #scikitplot
    wandb.log({"confusion_matrix": wandb.Image(fig)}, commit=False)

    # Metricas
    wandb.log({'pr': wandb.plots.precision_recall(y_test,predictions, labels=CATEGORIES)})
    #wandb.log({"f_beta_w":wandb.plot.fbeta_score(y_test, y_test_pred_class,average='weighted',beta=0.5)})


Sweep config (Definição dos parametros a serem otimizados e suas faixas)

In [ ]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 50
  },
  'parameters': {
        'epochs': {
            'distribution': 'int_uniform',
            'max': 160,
            'min': 80
        },
        'batch_size': {
            'values': [4,8,12]
        },
        'n_layers': {
            'values': [2, 3, 4]
        },
        'filters_size_1': {
            'values': [4, 8]
        }, 
        'filters_size_2': {
            'values': [4, 8]
        },
        'kernel_size_1': {
            'values': [1,3,5]
        },
        'kernel_size_2': {
            'values': [1,3,5]
        },
        'p_dropout': {
            'values': [0.05,0.1,0.15,0.2,0.25,0.3]
        },
        'learning_rate':{
            'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
        },
        'siz_dense_1': {
            'values': [40,50,60,70,80,90,100,120]
        },
        'siz_dense_2': {
            'values': [10,20,30,40,50,60]
        }
  }
}

Inicializar um novo projeto no sweep config

In [ ]:
sweep_id = wandb.sweep(sweep_config, project=project_name)

Ativando o sweep

In [ ]:
wandb.agent(sweep_id, function=train)